In [178]:
from params import *
import re

In [184]:
def read_data(sheet_name, header=3):
    data = pd.read_excel(f'{RAW_DATA_DIR}/dwelling-stock/subnationaldwellingstockbytenure2021.xlsx',
                         sheet_name=sheet_name,
                         header=header)
    colnames = data.columns.to_list()[4:]
    print(colnames)
    data = data.melt(value_vars=colnames)
    # new_frame = data.filter(like='Local')
    # for name, values in data.items():
    #     matched = re.findall(r'([0-9]{4})(\s?([A-Za-z]{1,25}\s?){1,10})', name)
    #     if matched:
    #         print(matched)
    #         date, var_name = matched[0][0], matched[0][1]
    #         print(new_frame.to_csv())
    return data

In [185]:
data = read_data('1b')
#data
#data.to_csv(f'{DATA_DIR}/dwelling-stock/dwelling-stock.csv')

['2012 Owned Outright\xa0', '2012 Owned with Mortgage or Loan\xa0', '2012 Private Rent\xa0', '2012 Social Rent\xa0', '2012 Total Dwellings\xa0', '2013 Owned Outright\xa0', '2013 Owned with Mortgage or Loan\xa0', '2013 Private Rent\xa0', '2013 Social Rent\xa0', '2013 Total Dwellings\xa0', '2014 Owned Outright', '2014 Owned with Mortgage or Loan', '2014 Private Rent', '2014 Social Rent', '2014 Total Dwellings', '2015 Owned Outright', '2015 Owned with Mortgage or Loan', '2015 Private Rent', '2015 Social Rent', '2015 Total Dwellings', '2016 Owned Outright', '2016 Owned with Mortgage or Loan', '2016 Private Rent', '2016 Social Rent', '2016 Total Dwellings', '2017 Owned Outright', '2017 Owned with Mortgage or Loan', '2017 Private Rent', '2017 Social Rent', '2017 Total Dwellings', '2018 Owned Outright', '2018 Owned with Mortgage or Loan', '2018 Private Rent', '2018 Social Rent', '2018 Total Dwellings', '2019 Owned Outright', '2019 Owned with Mortgage or Loan', '2019 Private Rent', '2019 Socia

In [186]:
data

,variable,value
0,2012 Owned Outright,11892
1,2012 Owned Outright,16170
2,2012 Owned Outright,20687
3,2012 Owned Outright,24908
4,2012 Owned Outright,14669
...,...,...
15395,2021 Total Dwellings,43327
15396,2021 Total Dwellings,53113
15397,2021 Total Dwellings,56222
15398,2021 Total Dwellings,79838
